In [1]:
import sys
import os

# Get the current directory
current_directory = os.getcwd()

# Add the current directory to the Python path
sys.path.append(current_directory)
print(current_directory)

from spectraltools import Specimen_Collection, Spectrum, create_path_if_not_exists
from metrics import *
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import math
import re
import scipy
import tensorflow as tf
import seaborn as sns

C:\Users\esteb\cicima\escarabajos\gamma


In [2]:
"""This section allows the user to choose their workplace location.
This is important if the user has multiple locations and operating systems in which this 
script is run"""

#select location
working_at = "wfh"

#Training data is used when we are already certain of species and genera for a particular sample
training_data_is_used = False

if working_at == "colaboratory":
  from google.colab import drive
  drive.mount("/content/drive")
  #base folder
  """Select the location for your base folder"""
    
  base_folder = r"/content/drive/My Drive/CICIMA/escarabajos_files"
  
elif working_at == "wfh":

    """Select the location of your base folder"""
    base_folder = r"C:\Users\esteb\cicima\escarabajos"

elif working_at == "cicima_desktop":
  
    """Select the location of your base folder"""
    base_folder = r"C:\Users\EstebanSoto\Jupyter\escarabajos"

elif working_at == "cicima_laptop":
    
    """Select the location of your base folder"""
    base_folder = r"/home/vinicio/escarabajos"

#define the location of the tables with information about the collections and its parent directory

collection_tables_main_path =  os.path.join(base_folder, "L1050_data","collections")
collection_files_main_path = os.path.join(base_folder, "L1050_data")

# Define report location
report_location = os.path.join(base_folder, "reports","data_analysis")

#collection_descriptor = r"CICIMAUCR and ANGSOL" tododelete

#File location and metadata location for collection 1
angsol_collection_path = os.path.join(collection_files_main_path,"ANGSOL","average") 
angsol_collection_metadata = os.path.join(collection_tables_main_path,"CICIMA-beetles-general-inventory - ANGSOL.txt") 

#File location and metadata location for collection 2
cicimaucr_collection_path = os.path.join(collection_files_main_path,r"TRA_data_CICIMA_INBUCR","CICIMAUCR","reflectance")  #listo
cicimaucr_collection_2_path = os.path.join(collection_files_main_path,r"CICIMA-2024-01-REFLECTANCE","average")
cicimaucr_collection_3_path = os.path.join(collection_files_main_path,r"CICIMA-2024-03-REFLECTANCE","without iris nor lens","average")
cicimaucr_collection_4_path = os.path.join(collection_files_main_path,r"CICIMA-2024-05-REFLECTANCE","average")
cicima_ucr_metadata = os.path.join(collection_tables_main_path,r"CICIMA-beetles-general-inventory - CICIMAUCR.txt") 

#File location and metadata location for collection 3
inbucr_collection_path = os.path.join(collection_files_main_path,r"INBUCR","average") #listo
inbucr_metadata = os.path.join(collection_tables_main_path,r"CICIMA-beetles-general-inventory - INBUCR.txt") 

#File location and metadata location for collection 4
bioucr_collection_path = os.path.join(collection_files_main_path,r"BIOUCR","average")  #listo
bioucr_metadata = os.path.join(collection_tables_main_path,r"CICIMA-beetles-general-inventory - BIOUCR.txt") 

#agregated data location, here averages and std will be saved when training data and retreived when classifying spectra
agregated_data_location = os.path.join(base_folder, "aggregated_data")

In [3]:
#Collections
angsol_collection = Specimen_Collection("ANGSOL", angsol_collection_path, angsol_collection_metadata, "HIGH")
angsol_collection.set_description("ANGSOL collection has specimens that belong to Angel Solís. The confidence that we have about specimen identification is high.")

cicimaucr_collection = Specimen_Collection("CICIMAUCR1", cicimaucr_collection_path, cicima_ucr_metadata, "HIGH")
cicimaucr_collection_2 = Specimen_Collection("CICIMAUCR2", cicimaucr_collection_2_path, cicima_ucr_metadata, "HIGH")
cicimaucr_collection_3 = Specimen_Collection("CICIMAUCR3", cicimaucr_collection_3_path, cicima_ucr_metadata, "HIGH")
cicimaucr_collection_3.set_description("""The most part of CICIMA specimens belongs to this collecttion""")

cicimaucr_collection_4 = Specimen_Collection("CICIMAUCR4", cicimaucr_collection_4_path, cicima_ucr_metadata, "HIGH")
cicimaucr_collection_4.set_description("""This collection has 3 kalinini specimens which were not used in training. 
                                        These are intended to be used as test subjects""")

inbucr_collection = Specimen_Collection("INBUCR", inbucr_collection_path, inbucr_metadata, "MID")
bioucr_collection = Specimen_Collection("BIOUCR", bioucr_collection_path, bioucr_metadata, "LOW")

collection_list = [
                    #angsol_collection,
                    cicimaucr_collection,
                    cicimaucr_collection_2,
                    cicimaucr_collection_3,
                    #inbucr_collection,
                    #bioucr_collection,
                    ]
collection_names_set = set([collection.name for collection in collection_list])
collection_names = " ".join( sorted(collection_names_set))

prediction_list = [
                    #angsol_collection,
                    #cicimaucr_collection,
                    #cicimaucr_collection_2,
                    #cicimaucr_collection_3,
                    cicimaucr_collection_4,
                    #inbucr_collection,
                    #bioucr_collection,
                    ]
prediction_collection_names_set = set([collection.name for collection in collection_list])
prediction_collection_names = " ".join( sorted(collection_names_set))


#print(collection_names)
#date
from datetime import datetime
current_date = datetime.now().date()

def get_filtered_spectra(collection_list):

    all_spectra = []
    
    for collection in collection_list:
        all_spectra += collection.get_spectra()

    all_spectra = [item for item in all_spectra if item.get_species() in ["kalinini", "resplendens", "cupreomarginata"]]
    return all_spectra
    
def get_spectra(collection_list):

    all_spectra = []
    
    for collection in collection_list:
        all_spectra += collection.get_spectra()

    return all_spectra  
    
training_spectra = get_filtered_spectra(collection_list)
prediction_spectra  = get_spectra(prediction_list) 

for spectrum in prediction_spectra:
    print(spectrum.get_species())

training_spectra

nan
nan
nan


In [4]:
def pad_list(lst, filler=0):
    metrics = lst[1]
    
    for metric in metrics:
        #print(metric)
        
        max_length = max([len(list) for list in metrics])
        
        #print(max_length)
        
        padded_list = []

        for element in metrics:

            padded_sublist = element
            
            padded_sublist += [filler] * (max_length - len(padded_sublist))
            padded_list.append(padded_sublist)
            #print(padded_sublist)
    final_list = [lst[0], np.array(padded_list), lst[2]]
    return final_list

In [5]:
### Training data 

#scalar
gamma_arbitrary_limits_data = feature_and_label_extractor(Gamma_Arbitrary_Limits, training_spectra)
gamma_first_two_peaks_data = feature_and_label_extractor(Gamma_First_Two_Peaks, training_spectra)
gamma_area_under_curve_data = feature_and_label_extractor(Gamma_Area_Under_Curve_Naive, training_spectra)
gamma_area_under_curve_first_min_cut_data = feature_and_label_extractor(Gamma_Area_Under_Curve_First_Min_Cut, training_spectra)

#vectorial
gamma_vector_relative_reflectance_data = feature_and_label_extractor(Gamma_Vector_Relative_Reflectance, training_spectra)
wavelength_vector_data = feature_and_label_extractor(Wavelength_Vector, training_spectra)
critical_points_data = feature_and_label_extractor(Critical_Points, training_spectra)
                                                  
maximum_points_data = feature_and_label_extractor(Maximum_Points, training_spectra)
minimum_points_data = feature_and_label_extractor(Minimum_Points, training_spectra)
maximum_points_normalized_data = feature_and_label_extractor(Maximum_Points_Normalized, training_spectra)
minimum_points_normalized_data = feature_and_label_extractor(Minimum_Points_Normalized, training_spectra)


In [14]:
minimum_points_data

[['CICIMAUCR0104',
  'CICIMAUCR0105',
  'CICIMAUCR0158',
  'CICIMAUCR0001',
  'CICIMAUCR0006',
  'CICIMAUCR0008',
  'CICIMAUCR0009',
  'CICIMAUCR0012',
  'CICIMAUCR0013',
  'CICIMAUCR0014',
  'CICIMAUCR0015',
  'CICIMAUCR0018',
  'CICIMAUCR0019',
  'CICIMAUCR0020',
  'CICIMAUCR0021',
  'CICIMAUCR0070',
  'CICIMAUCR0071',
  'CICIMAUCR0097',
  'CICIMAUCR0098',
  'CICIMAUCR0100',
  'CICIMAUCR0101',
  'CICIMAUCR0104',
  'CICIMAUCR0108',
  'CICIMAUCR0112',
  'CICIMAUCR0113',
  'CICIMAUCR0116',
  'CICIMAUCR0141',
  'CICIMAUCR0158'],
 [[array([ 586.,  755., 1385.]), array([75.594599, 80.225793, 33.293938])],
  [array([ 586.,  776., 1396., 1668.]),
   array([63.605839, 82.017313, 33.658805, 27.925045])],
  [array([ 572.,  742., 1113., 1213.]),
   array([61.776244, 52.898355, 21.963665, 21.272537])],
  [array([ 567.,  765., 1123., 1300., 1649., 1848.]),
   array([65.361779, 88.156557, 67.663   , 67.7368  , 47.5637  , 43.790775])],
  [array([ 550.,  772.,  858., 1119., 1303., 1654., 1858.]),
   

In [6]:
#Save averages and std for training data
metrics_list = [Gamma_Arbitrary_Limits, Gamma_First_Two_Peaks, Gamma_Area_Under_Curve_Naive, Gamma_Area_Under_Curve_First_Min_Cut, 
                Gamma_Area_Under_Curve_First_Min_Cut]

for metric in metrics_list:
    save_aggregated_data(metric,training_spectra,agregated_data_location)


Directory 'C:\Users\esteb\cicima\escarabajos\aggregated_data\metric_avg_std' already exists.
Directory 'C:\Users\esteb\cicima\escarabajos\aggregated_data\metric_avg_std' already exists.
Directory 'C:\Users\esteb\cicima\escarabajos\aggregated_data\metric_avg_std' already exists.
Directory 'C:\Users\esteb\cicima\escarabajos\aggregated_data\metric_avg_std' already exists.
Directory 'C:\Users\esteb\cicima\escarabajos\aggregated_data\metric_avg_std' already exists.


In [7]:
#read averages and std
avg_std_location = os.path.join(agregated_data_location, "metric_avg_std")
dfs = read_aggregated_data(avg_std_location)
dfs

['Gamma_Arbitrary_Limits.txt', 'gamma_area_under_curve_cut_first_minimum.txt', 'Gamma_Area_Under_Curve_Naive.txt', 'Gamma_First_Two_Peaks.txt']


{'Gamma_Arbitrary_Limits.txt':            species      mean       std
 0  cupreomarginata  1.230355  0.627756
 1         kalinini  1.413579  0.168335
 2      resplendens  1.309400  0.283687,
 'gamma_area_under_curve_cut_first_minimum.txt':            species      mean       std
 0  cupreomarginata  3.756872  0.778950
 1         kalinini  2.598407  0.468501
 2      resplendens  2.334925  1.246243,
 'Gamma_Area_Under_Curve_Naive.txt':            species      mean       std
 0  cupreomarginata  2.205516  0.506670
 1         kalinini  1.872984  0.213630
 2      resplendens  1.934388  0.480075,
 'Gamma_First_Two_Peaks.txt':            species      mean       std
 0  cupreomarginata  1.029802  0.076122
 1         kalinini  1.339818  0.118268
 2      resplendens  1.133386  0.464311}

In [8]:
#similarity index

def similarity_index(spectrum, averages_df, standard_deviations_df , species):
    def distance(x, x0, sigma_0):
        index =  (((x - x0)**4)**(1/4))
        #print(f" x {x} - x0 {x0} = {(x - x0)} and index = {index}")
        return index
        #def get_gamma_factor(self, spectrum):
    #get first n peaks of the gamma
    n = 5
    peaklist = PeakList(spectrum)
    peaks = peaklist.get_peaks()[0:n]

    #loads species average values:

    averages_df = averages_df[averages_df["species"] == species]
    average_x_df = averages_df[["x0","x1","x2","x4","x5"]].values[0]

    standard_deviations_df = standard_deviations_df[standard_deviations_df["species"] == species]
    standard_deviation_x_df = standard_deviations_df[["x0","x1","x2","x4","x5"]].values[0]

    #print(f"averages_df: {averages_df}")
    #for each peak, calculate the distances to the averages x values
    similarity_index = 0.0

    for peak_i,xi_0, sigmai_0 in zip(peaks, average_x_df, standard_deviation_x_df):
        xi = peak_i.x_value
        similarity_index += distance(xi, xi_0, sigmai_0)

    #normalize for the number of points
    similarity_index = similarity_index/n
    #print(f"gamma: {gamma}")
    return similarity_index

In [9]:
# Prediction data

prediction_gamma_arbitrary_limits_data = feature_and_label_extractor(Gamma_Arbitrary_Limits, prediction_spectra)
prediction_gamma_first_two_peaks_data = feature_and_label_extractor(Gamma_First_Two_Peaks, prediction_spectra)
prediction_gamma_area_under_curve_data = feature_and_label_extractor(Gamma_Area_Under_Curve_Naive, prediction_spectra)
prediction_gamma_area_under_curve_first_min_cut_data = feature_and_label_extractor(Gamma_Area_Under_Curve_First_Min_Cut, prediction_spectra)
prediction_gamma_vector_relative_reflectance_data = feature_and_label_extractor(Gamma_Vector_Relative_Reflectance, prediction_spectra)
prediction_critical_points_data = feature_and_label_extractor(Critical_Points, prediction_spectra)


In [10]:




#print(critical_points_data)
pad_list(critical_points_data, filler = np.array([0,0]))
pad_list(gamma_vector_relative_reflectance_data)
data = pad_list(wavelength_vector_data)

data

[['CICIMAUCR0104',
  'CICIMAUCR0105',
  'CICIMAUCR0158',
  'CICIMAUCR0001',
  'CICIMAUCR0006',
  'CICIMAUCR0008',
  'CICIMAUCR0009',
  'CICIMAUCR0012',
  'CICIMAUCR0013',
  'CICIMAUCR0014',
  'CICIMAUCR0015',
  'CICIMAUCR0018',
  'CICIMAUCR0019',
  'CICIMAUCR0020',
  'CICIMAUCR0021',
  'CICIMAUCR0070',
  'CICIMAUCR0071',
  'CICIMAUCR0097',
  'CICIMAUCR0098',
  'CICIMAUCR0100',
  'CICIMAUCR0101',
  'CICIMAUCR0104',
  'CICIMAUCR0108',
  'CICIMAUCR0112',
  'CICIMAUCR0113',
  'CICIMAUCR0116',
  'CICIMAUCR0141',
  'CICIMAUCR0158'],
 array([[ 586.,  755., 1385.,    0.,    0.,    0.,    0.,    0.,    0.],
        [ 586.,  776., 1396., 1668.,    0.,    0.,    0.,    0.,    0.],
        [ 572.,  742., 1113., 1213.,    0.,    0.,    0.,    0.,    0.],
        [ 567.,  765., 1123., 1300., 1649., 1848.,    0.,    0.,    0.],
        [ 550.,  772.,  858., 1119., 1303., 1654., 1858.,    0.,    0.],
        [ 573.,  742.,  858., 1118., 1304.,    0.,    0.,    0.,    0.],
        [ 577.,  797.,  858.,

In [11]:
import sys

print(sys.version_info)

from tensorflow.keras import layers

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)

def replace_strings_3(lst):
    mapping = {"kalinini": 0, "resplendens": 1, "cupreomarginata": 2}
    return [mapping.get(item, item) for item in lst]

def replace_species_with_categorical(df):
    
    df.loc[df["species"]=="kalinini","species"] = 0
    df.loc[df["species"]=="resplendens", "species"] = 1
    df.loc[df["species"]=="cupreomarginata", "species"] = 2
    
    return df
    
def plot_curve(epochs, hist, list_of_metrics):
  """Plot a curve of one or more classification metrics vs. epoch."""  
  # list_of_metrics should be one of the names shown in:
  # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Loaded the plot_curve function.")

def transpose_list(lst):
    return list(zip(*lst))
    
def get_nth_feature(data, n):
    feature_vector = [data[0], [x[n] for x in data[1]] , data[2]] 
    return feature_vector
def scatter_plot_2_variables(df_1, df_2):

    joint_df = pd.merge(df_1, df_2, on=["code", "species"], how="inner")
    column_list = joint_df.columns.tolist()
    print(column_list)
    column_list = [x for x in column_list if x not in ["code", "species"] ]
    print(column_list)
    plt.figure()
    sns.scatterplot(joint_df, x=column_list[0], y =column_list[1], hue="species")
    plt.show()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


def scatter_plot_3_variables(df_1, df_2, df_3):
    
    joint_df = pd.merge(df_1, df_2, on=["code", "species"], how="inner")
    joint_df = pd.merge(joint_df, df_3, on=["code", "species"], how="inner")
    
    column_list = joint_df.columns.tolist()
    print(column_list)
    column_list = [x for x in column_list if x not in ["code", "species"] ]
    print(column_list)
    
    x = joint_df[column_list[0]]
    y = joint_df[column_list[1]]
    z = joint_df[column_list[2]]
    species = joint_df["species"]
    
    # Create color map
    colors = {'kalinini': 'r', 'resplendens': 'g', 'cupreomarginata': 'b'}
    
    # Create figure and 3D axis
    fig = plt.figure(figsize=(15, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot points with color based on the fourth dimension
    for category in set(species):
        indices = species == category
        ax.scatter(x[indices], y[indices], z[indices], c=colors[category], label=category, marker='o')
    
    # Set labels and title
    ax.set_xlabel(f'{column_list[0]}')
    ax.set_ylabel(f'{column_list[1]}')
    ax.set_zlabel(f'{column_list[2]}')
    #ax.set_title('3D Scatter plot with species based on Fourth Dimension')
    
    # Add legend
    ax.legend()
    
    plt.show()

sys.version_info(major=3, minor=9, micro=18, releaselevel='final', serial=0)
Loaded the plot_curve function.


In [12]:
#Training data. Define dataframees

gal_df = pd.DataFrame(transpose_list(gamma_arbitrary_limits_data), columns =["code", "g_arbitrary_limits", "species"]) 
gftp_df = pd.DataFrame(transpose_list(gamma_first_two_peaks_data), columns =["code", "g_first_2_peaks", "species"])
gaucfmc_df = pd.DataFrame(transpose_list(gamma_area_under_curve_first_min_cut_data), columns =["code", "g_area_und_curve_first_min", "species"]) 
gauc_df = pd.DataFrame(transpose_list(gamma_area_under_curve_data), columns =["code", "g_area_und_curve", "species"])

complete_df = pd.merge(gal_df, gftp_df, on=["species","code"], how="inner")
complete_df = pd.merge(complete_df, gauc_df, on=["species","code"], how="inner")
complete_df = pd.merge(complete_df, gaucfmc_df, on=["species","code"], how="inner")
complete_df.drop(columns=['code'], inplace=True)

complete_df = replace_species_with_categorical(complete_df)
#complete_df.drop(columns=['species'], inplace=True)
complete_df

shuffled_df = complete_df.sample(frac=1, random_state=42)  # Random_state for reproducibility

# Define the fraction of data to be used for training
train_fraction = 0.66  # For example, 80% for training, 20% for testing

# Calculate the number of rows for the training set
train_size = int(train_fraction * len(complete_df))

# Split the shuffled DataFrame into train and test sets
train_data = shuffled_df.iloc[:train_size]
test_data = shuffled_df.iloc[train_size:]

# Split the shuffled DataFrame into features and labels
train_data_features = train_data.drop(columns=['species'], inplace=False)
train_data_labels =  train_data.drop(columns=['g_arbitrary_limits','g_first_2_peaks','g_area_und_curve_first_min','g_area_und_curve'], inplace=False)
test_data_features =  test_data.drop(columns=['species'], inplace=False)
test_data_labels =  test_data.drop(columns=['g_arbitrary_limits','g_first_2_peaks','g_area_und_curve_first_min','g_area_und_curve'], inplace=False)

from tensorflow.keras.utils import to_categorical

# Assuming y_train and y_test are your integer labels
train_data_labels_one_hot = to_categorical(train_data_labels, num_classes=2)
test_data_labels_one_hot = to_categorical(test_data_labels, num_classes=2)

print(train_data[train_data_labels["species"]==2])
print(train_data_features[train_data_labels["species"]==0])
#print(len(train_data))
#print(len(test_data))
#print(test_features)
#print(test_labels)

# Select 2
train_data_features = train_data_features.drop(columns=['g_arbitrary_limits',"g_area_und_curve"], inplace=False)
test_data_features =  test_data_features.drop(columns=['g_arbitrary_limits',"g_area_und_curve"], inplace=False)

print(train_data_labels[train_data_labels["species"]==0])
print(train_data_labels_one_hot[train_data_labels["species"]==0])

IndexError: index 2 is out of bounds for axis 1 with size 2

In [ ]:
def create_model_3(my_learning_rate):
  """Create and compile a deep neural net."""
  
  # All models in this course are sequential.
  model = tf.keras.models.Sequential()

  # The features are stored in a two-dimensional 28X28 array. 
  # Flatten that two-dimensional array into a one-dimensional 
  # 784-element array.
  model.add(tf.keras.layers.Flatten(input_shape=(2,)))

  # Define the first hidden layer.   
  model.add(tf.keras.layers.Dense(units=8, activation='relu'))

  # Define the first hidden layer.   
  model.add(tf.keras.layers.Dense(units=4, activation='relu'))

  # Define the first hidden layer.   
  model.add(tf.keras.layers.Dense(units=4, activation='relu'))
  
  # Define a dropout regularization layer. 
  model.add(tf.keras.layers.Dropout(rate=0.1))

  # Define the output layer. The units parameter is set to 10 because
  # the model must choose among 10 possible output values (representing
  # the digits from 0 to 9, inclusive).
  #
  # Don't change this layer.
  model.add(tf.keras.layers.Dense(units=2, activation='softmax'))     
                           
  # Construct the layers into a model that TensorFlow can execute.  
  # Notice that the loss function for multi-class classification
  # is different than the loss function for binary classification.  
  #model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),loss="sparse_categorical_crossentropy", metrics=['accuracy'])
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model    


def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):
  """Train the model by feeding it data."""

  history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True)
                      #validation_split=validation_split)
 
  # To track the progression of training, gather a snapshot
  # of the model's metrics at each epoch. 
  epochs = history.epoch
  hist = pd.DataFrame(history.history)

  return epochs, hist    

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.007
epochs = 160
batch_size = 3
validation_split = 0.05

# Establish the model's topography.
my_model = create_model_3(learning_rate)

# Train the model on the normalized training set.
#epochs, hist = train_model(my_model, x_train, y_train, 
                           #epochs, batch_size, validation_split)
epochs, hist = train_model(my_model, train_data_features, train_data_labels_one_hot, 
                           epochs, batch_size, validation_split)

# Plot a graph of the metric vs. epochs.
list_of_metrics_to_plot = ['accuracy']
plot_curve(epochs, hist, list_of_metrics_to_plot)

# Evaluate against the test set.
print("\n Evaluate the new model against the test set:")
my_model.evaluate(x=test_data_features, y=test_data_labels_one_hot, batch_size=batch_size)

### Notas: 
Mejores resultados usando el valor de batch size  4 o 3, acc 0.7
 Para 5, acc baja a 0.6
 Learning rate, buenos resultados en: 0.008-0.009. Acc 0.7
aprendizaje inestable: Cambiamos learning rate a 0.007 y epochs a 600
 Pasar regularización de 0 a 0.1 hace que el aprendizaje sea más suave
 Pasar de 4,4 a 8,4 hace el aprendizaje mas suave
 Pasar de 4,4 a 8,6 hace el aprendizaje menos efectivo
 Pasar de 4,4 a 8,4,4 hace el aprendizaje mas suave
aumentar el validation split de 0.1 a 0.3 reduce el acc a 0.3 0.6 0.6
bajar el validation split 0.1 a 0.05 reduce el acc a 0.5 0.6 0.7
dejar el val. split en 0.1 deja acc en 0.5 0.6 0.65

In [ ]:
#Save model 

# Define the file path where you want to save your model
model_file_path = 'trained_model_4_var.h5'

# Save the model
my_model.save(model_file_path)

print("Model saved successfully at:", model_file_path)

In [ ]:
# Load the saved model
model_file_path = 'trained_model_4_var.h5'
loaded_model2 = tf.keras.models.load_model(model_file_path)

#load prediction data

#Define dataframe
pred_gal_df = pd.DataFrame(transpose_list(prediction_gamma_arbitrary_limits_data), columns =["code", "g_arbitrary_limits", "real_species"]) 
pred_gftp_df = pd.DataFrame(transpose_list(prediction_gamma_first_two_peaks_data), columns =["code", "g_first_2_peaks", "real_species"])
pred_gauc_df = pd.DataFrame(transpose_list(prediction_gamma_area_under_curve_data), columns =["code", "g_area_und_curve", "real_species"])
pred_gaucfmc_df = pd.DataFrame(transpose_list(prediction_gamma_area_under_curve_first_min_cut_data), columns =["code", "g_area_und_curve_first_min", "real_species"]) 


#drop species

drop_pred_gal_df = pred_gal_df.copy()
drop_pred_gal_df.drop(columns=[ "real_species"], inplace=True)

drop_pred_gftp_df = pred_gftp_df.copy()
drop_pred_gftp_df.drop(columns=[ "real_species"], inplace=True)

drop_pred_gauc_df= pred_gauc_df.copy()
drop_pred_gauc_df.drop(columns=[ "real_species"], inplace=True)

drop_pred_gaucfmc_df=pred_gaucfmc_df.copy()
drop_pred_gaucfmc_df.drop(columns=[ "real_species"], inplace=True)


#merge on code
pred_complete_df = pd.merge(drop_pred_gal_df, drop_pred_gftp_df, on=["code"], how="inner")
pred_complete_df = pd.merge(pred_complete_df, drop_pred_gauc_df, on=["code"], how="inner")
pred_complete_df = pd.merge(pred_complete_df, drop_pred_gaucfmc_df, on=["code"], how="inner")

no_code_pred_complete_df= pred_complete_df.copy()
no_code_pred_complete_df.drop(columns=[ "code"], inplace=True)
pred_complete_df

In [ ]:
# Select 2
no_code_pred_complete_df = no_code_pred_complete_df.drop(columns=['g_arbitrary_limits',"g_area_und_curve"], inplace=False)

print(no_code_pred_complete_df)


In [ ]:

prediction_features = pd.DataFrame({name:np.array(value) for name, value in no_code_pred_complete_df.items()})

print(prediction_features)
#print(pred_complete_df)
#convert prediction_features to tensor
for element in prediction_features:
    prediction_features[element] = tf.convert_to_tensor(np.array(prediction_features[element]), dtype=tf.int64) 


# Predict using the loaded model
predictions = loaded_model2.predict(prediction_features)

# Print the predictions
#print(predictions)
prediction_df = pd.DataFrame(predictions, columns=["kalinini", "resplendens"])
prediction_df

In [ ]:
merged_prediction_data = pd.merge(pred_gal_df, pred_gftp_df , on=["code","real_species"], how="inner")
merged_prediction_data = pd.merge(merged_prediction_data, pred_gauc_df , on=["code","real_species"], how="inner")
merged_prediction_data = pd.merge(merged_prediction_data, pred_gaucfmc_df , on=["code","real_species"], how="inner")
merged_prediction_data